In [87]:
import datetime
import os
import sys
import numpy as np
from osgeo import gdal
import netCDF4
from utils import file_tools, gdal_tools

import geotrans_pack

In [88]:
nc_file = r'E:\DATA\a1_DATA_COLLECTION\a1_Hydrology\PRE\ERA5_origin\adaptor.mars.internal-1685862681.575161-3997-12-587de292-ff6e-4c7b-9d8e-63c0ed32768c.nc'

nc_obj = netCDF4.Dataset(nc_file)

In [89]:
# def hoursince2dt(hour_since_time):
#     base_time = datetime.datetime(1900, 1, 1, 0, 0, 0)
#     time_delta = datetime.timedelta(hours=hour_since_time)
#     target_time = base_time + time_delta
#     return target_time

def hoursince2dt(hours_since_1900):
    base_time = datetime.datetime(1900, 1, 1, 0, 0, 0)
    time_delta = np.timedelta64(1, 'h') * hours_since_1900
    target_time = np.datetime64(base_time) + time_delta

    return target_time

In [90]:
# :units = "hours since 1900-01-01 00:00:00.0";

nc_time_series = hoursince2dt(np.array(nc_obj['time']))



In [91]:
def save_tiff(output_path, data, projection, geotransform):
    """
    save array data to tiff
    :param output_path: full tif output path
    :param data: 2d data array
    :param projection:
    :param geotransform:
    :return:
    """
    driver = gdal.GetDriverByName('GTiff')
    # default single band
    bands = 1
    # set geotrans and proj
    rows, cols = data.shape

    dataset = driver.Create(output_path, cols, rows, bands, gdal.GDT_Float32)
    dataset.SetGeoTransform(geotransform)
    dataset.SetProjection(projection)

    # write data to tiff
    band = dataset.GetRasterBand(1)
    band.WriteArray(data.astype(np.float32))
    band.FlushCache()

proj = gdal_tools.wkt_proj(4326)
geotrans = (0, 0.25, 0.0, 90.0, 0.0, -0.25)



In [92]:
output_dir = r'D:\CODE\c1_py\b1_nc2tiff\era5_out'

time_flag = nc_time_series[0].astype(datetime.datetime).strftime('%Y%m%d')

merge_mat = np.zeros((721, 1440))


nc_data_mat = nc_obj['tp']
for _mat, _time  in zip(nc_data_mat, nc_time_series):

    # print(_mat.shape)
    current_date = _time.astype(datetime.datetime).strftime('%Y%m%d')

    if time_flag == current_date:
        merge_mat = merge_mat + _mat.data
        print(f'{current_date}  MERGE')
    else:
        save_tiff(
            os.path.join(output_dir, f"ERA5_{time_flag}.tif"),
            merge_mat,
            proj,
            geotrans,
        )
        # reset temp mat
        merge_mat = _mat.data

        print(f'{current_date}  SAVE')

    time_flag = current_date

20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200101  MERGE
20200102  SAVE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200102  MERGE
20200103  SAVE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  MERGE
20200103  